# [Reference](https://www.kaggle.com/code/kaggle5daysofai/day-2b-agent-tools-best-practices)

In [1]:
import os
from kaggle_secrets import UserSecretsClient

GEMINI_API_KEY = UserSecretsClient().get_secret("GEMINI_API_KEY")
os.environ["GOOGLE_API_KEY"] = GEMINI_API_KEY

In [2]:
import uuid
from google.genai import types

from google.adk.agents import LlmAgent
from google.adk.models.google_llm import Gemini
from google.adk.runners import Runner
from google.adk.sessions import InMemorySessionService

from google.adk.tools.mcp_tool.mcp_toolset import McpToolset
from google.adk.tools.tool_context import ToolContext
from google.adk.tools.mcp_tool.mcp_session_manager import StdioConnectionParams
from mcp import StdioServerParameters

from google.adk.apps.app import App, ResumabilityConfig
from google.adk.tools.function_tool import FunctionTool

In [3]:
retry_config = types.HttpRetryOptions(
    attempts=5,
    exp_base=7,
    initial_delay=1,
    http_status_codes=[429, 500, 503, 504]
)

In [4]:
mcp_image_server = McpToolset(
    connection_params = StdioConnectionParams(
        server_params = StdioServerParameters(
            command="npx",
            args=[
                "-y",
                "@modelcontextprotocol/server-everything"
            ],
            tool_filter=["getTinyImage"]
        ),
        timeout=30,
    )
)

In [5]:
image_agent = LlmAgent(
    name="ImageAgent",
    model = Gemini(
        model = "gemini-2.5-flash-lite"
    ),
    instruction="""
        Use MCP tool to generate funny image for the given queries
    """,
    tools = [mcp_image_server]
)

In [6]:

from google.adk.runners import InMemoryRunner

async with InMemoryRunner(agent=image_agent) as runner:
    response = await runner.run_debug(
        """Provide a sample tiny image""", verbose=True
    )


 ### Created new session: debug_session_id

User > Provide a sample tiny image


/usr/local/lib/python3.11/dist-packages/google/adk/tools/mcp_tool/mcp_tool.py:101: UserWarning: [EXPERIMENTAL] BaseAuthenticatedTool: This feature is experimental and may change or be removed in future versions without notice. It may introduce breaking changes at any time.
  super().__init__(


ImageAgent > [Calling tool: getTinyImage({})]
ImageAgent > [Tool result: {'content': [{'type': 'text', 'text': 'This is a tiny image:'}, {'type': 'image', 'data': 'iVBORw0KG...]


ImageAgent > This is a tiny image:
The image above is the MCP tiny image.


In [7]:
print(response)

[Event(model_version='gemini-2.5-flash-lite', content=Content(
  parts=[
    Part(
      function_call=FunctionCall(
        args={},
        id='adk-17074634-ac40-4807-aac3-5df9fd8edec9',
        name='getTinyImage'
      )
    ),
  ],
  role='model'
), grounding_metadata=None, partial=None, turn_complete=None, finish_reason=<FinishReason.STOP: 'STOP'>, error_code=None, error_message=None, interrupted=None, custom_metadata=None, usage_metadata=GenerateContentResponseUsageMetadata(
  candidates_token_count=10,
  prompt_token_count=646,
  prompt_tokens_details=[
    ModalityTokenCount(
      modality=<MediaModality.TEXT: 'TEXT'>,
      token_count=646
    ),
  ],
  total_token_count=656
), live_session_resumption_update=None, input_transcription=None, output_transcription=None, avg_logprobs=None, logprobs_result=None, cache_metadata=None, citation_metadata=None, invocation_id='e-1545a04d-0521-45f4-85d3-df7d0941cbd5', author='ImageAgent', actions=EventActions(skip_summarization=None, sta

In [8]:
from IPython.display import display, Image as IPImage
import base64

for event in response:
    if event.content and event.content.parts:
        for part in event.content.parts:
            if hasattr(part, "function_response") and part.function_response:
                for item in part.function_response.response.get("content", []):
                    if item.get("type") == "image":
                        display(IPImage(data=base64.b64decode(item["data"])))

In [66]:
LARGE_ORDER_THRESHOLD = 5

def place_shipping_order(num_containers: int, destination: str, tool_context: ToolContext) -> dict:
    if num_containers <= LARGE_ORDER_THRESHOLD:
        return {
            "staus": "approved",
            "order_id": f"ORD-{num_containers}-AUTO",
            "num_containers": num_containers,
            "message": f"Order auto-approved: {num_containers} containers to {destination}",
        }
    if not tool_context.tool_confirmation:
        tool_context.request_confirmation(
            hint = f"⚠️ Large order: {num_containers} containers to {destination}. Do you want to approve?",
            payload = {
                "num_containers": num_containers,
                "destination": destination
            },
        )
        return {
            "status": "pending",
            "message": f"Order for {num_containers} containers requires approval",
        }

    if tool_context.tool_confirmation.confirmed:
        return {
            "status": "approved",
            "order_id": f"ORD-{num_containers}-HUMAN",
            "num_containers": num_containers,
            "destination": destination,
            "message": f"Order approved: {num_containers} containers to {destination}",
        }
    else:
        return {
            "status": "rejected",
            "message": f"Order rejected: {num_containers} containers to {destination}",
        }

In [67]:
shipping_agent = LlmAgent(
    name="ShippingAgent",
    model=Gemini(
        model="gemini-2.5-flash-lite",
        retry_options=retry_config,
    ),
    instruction="""
    You are a shipping coordinator assistant.
  
      When users request to ship containers:
       1. Use the place_shipping_order tool with the number of containers and destination
       2. If the order status is 'pending', inform the user that approval is required
       3. After receiving the final result, provide a clear summary including:
          - Order status (approved/rejected)
          - Order ID (if available)
          - Number of containers and destination
       4. Keep responses concise but informative
    """,
    tools=[FunctionTool(place_shipping_order)]
)

In [68]:
shipping_app = App(
    name="shipping_coordinator",
    root_agent=shipping_agent,
    resumability_config=ResumabilityConfig(is_resumable=True),
)

/tmp/ipykernel_117/2323755241.py:4: UserWarning: [EXPERIMENTAL] ResumabilityConfig: This feature is experimental and may change or be removed in future versions without notice. It may introduce breaking changes at any time.
  resumability_config=ResumabilityConfig(is_resumable=True),


In [69]:
session_service= InMemorySessionService()

shipping_runner = Runner(
    app=shipping_app,
    session_service=session_service
)

In [70]:
def check_for_approval(events):
    for event in events:
        if event.content and event.content.parts:
            for part in event.content.parts:
                if(
                    part.function_call and part.function_call.name=="adk_request_confirmation"
                ):
                    return {
                        "approval_id": part.function_call.id,
                        "invocation_id": event.invocation_id,
                    }

    return None

In [71]:
def print_agent_response(events):
    for event in events:
        if event.content and event.content.parts:
            for part in event.content.parts:
                if part.text:
                    print(f"Agent > {part.text}")

In [80]:
def create_approval_response(approval_info, approved):
    confirmation_response = types.FunctionResponse(
        id=approval_info["approval_id"],
        name="adk_request_confirmation",
        response={
            "confirmed": approved
        },
    )

    return types.Content(
        role="user",
        parts=[types.Part(function_response=confirmation_response)]
    )

In [77]:
async def run_shipping_workflow(query: str, auto_approve: bool = True):
    print(f"\n{'='*60}")
    print(f"User > {query}\n")

    session_id = f"order_{uuid.uuid4().hex[:8]}"

    await session_service.create_session(
        app_name="shipping_coordinator", user_id="test_user", session_id=session_id
    )

    print(await session_service.list_sessions(user_id="test_user", app_name="shipping_coordinator"))

    query_content = types.Content(role="user", parts=[types.Part(text=query)])
    events=[]

    async for event in shipping_runner.run_async(
        user_id="test_user", session_id=session_id, new_message=query_content
    ):
        events.append(event)

    approval_info = check_for_approval(events)

    if approval_info:
        print(f"⏸️  Pausing for approval...")
        print(f"🤔 Human Decision: {'APPROVE ✅' if auto_approve else 'REJECT ❌'}\n")

        async for event in shipping_runner.run_async(
            user_id="test_user",
            session_id = session_id,
            new_message = create_approval_response(
                approval_info, auto_approve
            ),
            invocation_id=approval_info["invocation_id"]
        ):
            if event.content and event.content.parts:
                for part in event.content.parts:
                    if part.text:
                        print(f"Agent > {part.text}")
    else:
        print_agent_response(events)

In [78]:
# Demo 1: It's a small order. Agent receives auto-approved status from tool
await run_shipping_workflow("Ship 3 containers to Singapore")


User > Ship 3 containers to Singapore

sessions=[Session(id='order_82b118ba', app_name='shipping_coordinator', user_id='test_user', state={}, events=[], last_update_time=1765114896.824077), Session(id='order_2634db6f', app_name='shipping_coordinator', user_id='test_user', state={}, events=[], last_update_time=1765114897.42183), Session(id='order_08ed7a48', app_name='shipping_coordinator', user_id='test_user', state={}, events=[], last_update_time=1765114934.3147104)]


Agent > Your order of 3 containers to Singapore has been approved. The order ID is ORD-3-AUTO.


In [81]:
# Demo 2: Workflow simulates human decision: APPROVE ✅
await run_shipping_workflow("Ship 10 containers to Rotterdam", auto_approve=True)


User > Ship 10 containers to Rotterdam

sessions=[Session(id='order_82b118ba', app_name='shipping_coordinator', user_id='test_user', state={}, events=[], last_update_time=1765114896.824077), Session(id='order_2634db6f', app_name='shipping_coordinator', user_id='test_user', state={}, events=[], last_update_time=1765114897.42183), Session(id='order_08ed7a48', app_name='shipping_coordinator', user_id='test_user', state={}, events=[], last_update_time=1765114935.746846), Session(id='order_500f834d', app_name='shipping_coordinator', user_id='test_user', state={}, events=[], last_update_time=1765114937.664678), Session(id='order_cbc3dee5', app_name='shipping_coordinator', user_id='test_user', state={}, events=[], last_update_time=1765115111.7121334)]


/usr/local/lib/python3.11/dist-packages/google/adk/agents/invocation_context.py:298: UserWarning: [EXPERIMENTAL] BaseAgentState: This feature is experimental and may change or be removed in future versions without notice. It may introduce breaking changes at any time.
  self.agent_states[event.author] = BaseAgentState()


⏸️  Pausing for approval...
🤔 Human Decision: APPROVE ✅

Agent > Shipping order approved: 10 containers to Rotterdam. The order ID is ORD-10-HUMAN.


In [82]:
# Demo 3: Workflow simulates human decision: REJECT ❌
await run_shipping_workflow("Ship 8 containers to Los Angeles", auto_approve=False)


User > Ship 8 containers to Los Angeles

sessions=[Session(id='order_82b118ba', app_name='shipping_coordinator', user_id='test_user', state={}, events=[], last_update_time=1765114896.824077), Session(id='order_2634db6f', app_name='shipping_coordinator', user_id='test_user', state={}, events=[], last_update_time=1765114897.42183), Session(id='order_08ed7a48', app_name='shipping_coordinator', user_id='test_user', state={}, events=[], last_update_time=1765114935.746846), Session(id='order_500f834d', app_name='shipping_coordinator', user_id='test_user', state={}, events=[], last_update_time=1765114937.664678), Session(id='order_cbc3dee5', app_name='shipping_coordinator', user_id='test_user', state={}, events=[], last_update_time=1765115113.096531), Session(id='order_9896120b', app_name='shipping_coordinator', user_id='test_user', state={}, events=[], last_update_time=1765115124.4230099)]


⏸️  Pausing for approval...
🤔 Human Decision: REJECT ❌

Agent > The shipping order for 8 containers to Los Angeles has been rejected.
